## Imports

In [ ]:
import os
from glob import glob
from tqdm import tqdm

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.integrate import simpson
from scipy.optimize import curve_fit
from scipy.signal import argrelextrema
from scipy.stats import spearmanr, median_abs_deviation

from astropy.io import fits
from astropy.convolution import convolve, Gaussian1DKernel
from astropy.stats import sigma_clipped_stats

%matplotlib inline

In [ ]:
plt.rcParams.update({'font.size': 20})

## Load Lines and Data

In [ ]:
data_path = '../data'
lines_vac, lines_air = np.loadtxt(f'{data_path}/hydroxyl_lines_rousselot_2000.txt').T

In [ ]:
results_path = '../results'
df = pd.read_csv(f'{results_path}/lines_norm_gdl_fit.csv', index_col=0)
columns = df.columns

In [ ]:
col_sub = []
for column in columns[9:28]:
    if 'sigma1_cov' in column or 'sigma2_cov' in column:
        col_sub.append(column[9:])
    else:
        col_sub.append(column.split('_')[1])

In [ ]:
col_sub = np.array(col_sub)

In [ ]:
col_sub

In [ ]:
time_centered = df['spec_mjd'] - int(df['spec_mjd'].min())

In [ ]:
mask1_1 = (time_centered > 504) & (time_centered < 504.5)
mask1_2 = (time_centered > 504.58) & (time_centered < 505.515)
mask1_3 = (time_centered > 505.58) & (time_centered < 506.578)
mask1_4 = (time_centered > 506.61) & (time_centered < 507)
mask1 = mask1_1 | mask1_2 | mask1_3 | mask1_4
mask2 = (time_centered > 543) & (time_centered < 547)

In [ ]:
def measure_short_time_variability(df, lines_vac, mask, index):
    
    stats = []
    for line in lines_vac: 
        line_column = str(line) + '_sum'
        flux = df[line_column].values[mask]
        diff = (flux[1:flux.shape[0]:index] - flux[0:-1:index])
        diff_mean = np.nanmean(diff)
        diff_std = np.nanstd(diff)
        diff_med = np.nanmedian(diff)
        diff_mad = median_abs_deviation(diff, nan_policy='omit')
        stats.append([diff_mean, diff_std, diff_med, diff_mad])
    
    return np.array(stats)

In [ ]:
stats_5_1 = measure_short_time_variability(df, lines_vac, mask1, 1)
stats_5_2 = measure_short_time_variability(df, lines_vac, mask2, 1)
stats_10_1 = measure_short_time_variability(df, lines_vac, mask1, 2)
stats_10_2 = measure_short_time_variability(df, lines_vac, mask2, 2)
stats_25_1 = measure_short_time_variability(df, lines_vac, mask1, 5)
stats_25_2 = measure_short_time_variability(df, lines_vac, mask2, 5)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: 5 Minute Variability', fontsize=fs)
plt.errorbar(lines_vac/10000, stats_5_1[:, 0]+50, stats_5_1[:, 1], fmt='C5o', label='Event 1')
plt.errorbar(lines_vac/10000, stats_5_2[:, 0]-50, stats_5_2[:, 1], fmt='C7o', label='Event 2')
plt.xlabel('Wavelength ($\mu$m)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: 10 Minute Variability', fontsize=fs)
plt.errorbar(lines_vac/10000, stats_10_1[:, 0]+50, stats_10_1[:, 1], fmt='C5o', label='Event 1')
plt.errorbar(lines_vac/10000, stats_10_2[:, 0]-50, stats_10_2[:, 1], fmt='C7o', label='Event 2')
plt.xlabel('Wavelength ($\mu$m)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: 10 Minute Variability', fontsize=fs)
plt.scatter(lines_vac/10000, stats_10_1[:, 0]+2.5, color='C5', label='Event 1')
plt.scatter(lines_vac/10000, stats_10_2[:, 0]-2.5, color='C7', label='Event 2')
plt.xlabel('Wavelength ($\mu$m)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.legend(fontsize=fs)

In [ ]:
fs = 45
plt.figure(figsize=[30,10])
plt.grid(linewidth=2)
plt.title('SPIRou Sky: 5 Minute Variability', fontsize=fs)
plt.scatter(lines_vac/10000, stats_5_1[:, 0]+0.1, color='C5', label='Event 1')
plt.scatter(lines_vac/10000, stats_5_2[:, 0]-0.1, color='C7', label='Event 2')
plt.xlabel('Wavelength ($\mu$m)', fontsize=fs)
plt.ylabel('Relative Flux', fontsize=fs)
plt.tick_params(labelsize=fs)
plt.legend(fontsize=fs)